In [ ]:
%load_ext autoreload
%autoreload 2
import dynamics
import c_dynamics
import numpy as np
import matplotlib.pyplot as plt
import main
from scipy import optimize

In [ ]:
main.plotCoefficientsVsReynoldsNumber()

In [ ]:
main.plotCorrectionCoefficients()

In [ ]:
main.plotSettlingSpeedVsAspectRatio()

# Matlab comparison

In [ ]:
a_parallel=2.4e-05
a_perp=0.00012
g=9.82
nu=1.5115e-05
const = dynamics.SystemConstants(
    gravitational_acceleration=g,
    fluid_kinematic_viscosity=nu,
    a_perp=a_perp,
    a_para=a_parallel,
)

mat_data = np.loadtxt("traj_apara2.4e-05_aperp0.00012_lambda0.2_g9.82_nu1.5115e-05_curlyR996_taup0.042174_CF0.49114_CT0.73226.txt")
t_mat = mat_data[:,0]
res_mat = mat_data[:,1:]
initial = res_mat[0]
print(res_mat.shape)
print(f"beta: {const.beta}")
x0     = initial[0:3] * np.sqrt(const.tau_p * const.nu) / (g * const.tau_p ** 2)
v0     = initial[3:6] * np.sqrt(const.tau_p * const.nu) / (g * const.tau_p ** 2)
n0     = initial[6:9]
omega0 = initial[9:12]
y0 = np.concat([x0, v0, n0, omega0])
t_end = t_mat[-1]
t = np.linspace(0.0, t_end, num=1000)

t_py, res_py = c_dynamics.solveDynamics(
    y0=y0,
    const=const,
    t_eval=t,
    t_span=(0.0, t_end),
    rel_tol=1e-12,
    abs_tol=1e-12,
    event_type=0
)
res_py[:,:3] *= (np.linalg.norm(const.g) * const.tau_p ** 2) / np.sqrt(const.tau_p * const.nu)
res_py[:,3:6] *= (np.linalg.norm(const.g) * const.tau_p ** 2) / np.sqrt(const.tau_p * const.nu)
res_compatible = np.vstack([np.interp(t_mat, t, res_py[:,i]) for i in range(12)]).T
plt.plot(t_py, res_py[:,6+0], label="python x", color="black", ls="solid", lw=2)
plt.plot(t_py, res_py[:,6+1], label="python y", color="blue", ls="solid", lw=2)
plt.plot(t_py, res_py[:,6+2], label="python z", color="grey", ls="solid", lw=2)
plt.plot(t_mat, res_mat[:,6+0], label="matlab x", ls=":", color="orange", lw=1.5)
plt.plot(t_mat, res_mat[:,6+1], label="matlab y", ls=":", color="red", lw=1.5)
plt.plot(t_mat, res_mat[:,6+2], label="matlab z", ls=":", color="green", lw=1.5)
plt.legend()
# plt.ylim(-1, 2.0)
plt.xlabel("Time (1)")
plt.ylabel("Velocity (1)")
plt.show()

# Analytical bifurcation

In [ ]:
from dynamics import (
    shapeFactor,
    resistanceCoefficient,
    translationalResistanceCoefficients,
    rotationalResistanceCoefficients,
    computeCurlyAStokesForce,
    computeCurlyATorque
)

def hopfBifurcation(beta, curly_R, curly_V):
    A_perp, A_para = translationalResistanceCoefficients(beta)
    C_perp, C_para = rotationalResistanceCoefficients(beta)
    F_lambda = shapeFactor(beta)
    A_g = resistanceCoefficient(beta)
    c_config = c_dynamics.CppConfig(dynamics.SystemConstants(a_para=beta, a_perp=1.0))

    Re_p0 = max(1, beta) * curly_R * curly_V / (6 * np.pi * A_g)
    C_F = c_config.correctionFactorStokesForce(Re_p0)
    Re_p = (4 / 3) * max(1, beta) / A_g * (np.sqrt(1 + C_F * curly_R * curly_V / (4 * np.pi)) - 1) / C_F
    C_T = c_config.correctionFactorTorque(Re_p)

    curly_A_F = C_F * computeCurlyAStokesForce(curly_R, curly_V)
    curly_A_T = C_T * computeCurlyATorque(beta, curly_R, curly_V)
    v_g = (np.sqrt(1 + 8 * curly_A_F) - 1) / (4 * A_perp * curly_A_F)

    # curlyAT -> (APara CPerp lambda (1 + 3 APara curlyAF vG -
    #   APerp curlyAF vG) (5 CPerp lambda +
    #   3 APara^2 curlyAF (1 + lambda^2) vG -
    #   APara (1 + lambda^2) (-1 +
    #      APerp curlyAF vG)))/(vG^2 (10 CPerp lambda -
    #   APerp (1 + lambda^2) + 6 APara^2 curlyAF (1 + lambda^2) vG -
    #   2 APerp^2 curlyAF (1 + lambda^2) vG -
    #   2 APara (1 + lambda^2) (-1 + APerp curlyAF vG)))
    return (
        A_para*C_perp*beta*(1+3*A_para*curly_A_F*v_g-A_perp*curly_A_F*v_g)
        *(5*C_perp*beta+3*A_para**2*curly_A_F*(1+beta**2)*v_g-A_para*(1+beta**2)*(-1+A_perp*curly_A_F*v_g))
    ) / (v_g**2*(
        10*C_perp*beta-A_perp*(1+beta**2)
        +6*A_para**2*curly_A_F*(1+beta**2)*v_g
        -2*A_perp**2*curly_A_F*(1+beta**2)*v_g
        -2*A_para*(1+beta**2)*(-1+A_perp*curly_A_F*v_g)
    )) - curly_A_T

In [ ]:
x = np.logspace(-5, 5, num=1_000)
f_x = np.array([hopfBifurcation(0.2, 500, y) for y in x])

plt.plot(x, f_x)
plt.axhline(0, ls=":", lw=1, color="black")
plt.xscale("log")
# plt.yscale("log")
# plt.yscale("symlog")
plt.ylim(-1,1)

In [ ]:
def findRootCurlyV(beta: float, curly_R: float, guess_curly_V: float) -> float:
    try:
        curly_V = optimize.root_scalar(
            f=lambda x: hopfBifurcation(beta, curly_R, x),
            x0=guess_curly_V,
            maxiter=1000,
            xtol=1e-12,
            method='bisect',
            bracket=[guess_curly_V * 1e-2, guess_curly_V * 1e2]
        ).root
    except ValueError:
        curly_V = np.nan
    return curly_V

In [ ]:
g = 9.81
nu = 1.51147e-5
curly_R = 1.2e3 / 1.204
guess_curly_V = g * 1.44e-3 * 1e-9 / nu ** 2
beta_arr = np.linspace(0.01, 0.95, num=100)
curly_V_arr = np.array([findRootCurlyV(beta, curly_R, guess_curly_V) for beta in beta_arr])
volume_arr = curly_V_arr * nu ** 2 / g

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(beta_arr, volume_arr * 1e9 * 1e3, label="Analytical Bifurcation")
plt.xlabel("Aspect ratio $\\lambda$")
plt.ylabel("Particle volume (mm$^3\\times10^{-3}$)")
plt.legend()
plt.show()